In [13]:
import sys
!conda install --yes --prefix {sys.prefix} pandas=1.3.5
!conda install --yes --prefix {sys.prefix} plotly=5.5.0
!conda install --yes --prefix {sys.prefix} dash=2.0.0
!conda install --yes --prefix {sys.prefix} jupyter-dash=0.4.0
!pip install dash-bootstrap-components==1.0.2

^C
^C
^C
^C


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc

['Sun' 'Sat' 'Thur' 'Fri']


,total_bill,tip,sex,smoker,day,time,size
19,20.65,3.35,Male,No,Sat,Dinner,3
20,17.92,4.08,Male,No,Sat,Dinner,2
21,20.29,2.75,Female,No,Sat,Dinner,2
22,15.77,2.23,Female,No,Sat,Dinner,2
23,39.42,7.58,Male,No,Sat,Dinner,4
...,...,...,...,...,...,...,...
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2


In [2]:
df = pd.read_csv("DataWithElse.csv")
Plots = ['Box Plot', 'Bar Chart', 'Line Chart', 'Histogram', 'Scatter Plot', 'Sunburst Chart']
df['Year'] = df['Year'].astype(str)

In [3]:
# app = JupyterDash(__name__)
app = JupyterDash(
    external_stylesheets=[dbc.themes.BOOTSTRAP]
)

app.layout = html.Div([
    html.H4("Choose a dimension:"),
    html.Div([
        dbc.RadioItems(
            id="x-axis",
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-primary",
            labelCheckedClassName="active",
            options=[{'value': x, 'label': x} 
                for x in ['Year', 'Month', 'Trimestre', 'Continent', 'Country', 'Region', 'Industry', 'Category Type', 'Category Name', 'Skills', 'Employers', 'Jobs', 'Category Value']
            ],
            value='Category Value',
        ),
    ],
    className="radio-group a"),
    html.H4("Choose other dimension(s):"),
    html.Div([
        dbc.Checklist(
            id='y-axis',
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-success",
            labelCheckedClassName="active",
            options=[ {'value': x, 'label': x}
                for x in ['Year', 'Month', 'Trimestre', 'Continent', 'Country', 'Region', 'Industry', 'Category Type', 'Category Name', 'Skills', 'Employers', 'Jobs', 'Category Value']
            ],
            value=['Industry'],
        ),
    ],
    className="radio-group a"),
    html.Div([
        html.Div([
            html.H4("Filtered by:"),
            dcc.Dropdown(
                id="slct_filter_column", options=[],
                multi=False, value='', style={'width': "99%"}
            )],
            style={'width': "50%", 'float': 'left'}
        ),
        html.Div([
            html.H4("To be more specific:"),
            dcc.Dropdown(
                id="slct_specific_values", options=[],
                multi=True, value=[], style={'width': "99%"}
            )],
            style={'width': "50%", 'float': 'right'}
        )
    ], style={'display': 'flex'}, className="a"),
    html.Div([
        html.Div([
            html.H4("To choose an exact value(s):"),
            dcc.Dropdown(
                id="slct_exact_value", options=[],
                multi=True, value=[], style={'width': "99%"}
            )],
            style={'width': "50%", 'float': 'left'}
        ),
        html.Div([
            html.H4("Selecet the type of plot:"),
            dcc.Dropdown(
                id="slct_plot_type",
                options=[{'value': i, 'label': i} for i in Plots ],
                multi=False, value='Bar Chart', style={'width': "99%"}
            )],
            style={'width': "50%", 'float': 'left'}
        )
    ], style={'display': 'flex'}, className="a"),
    dcc.Graph(id="plot"),
], style={'padding': '1%'})

@app.callback(
    [
        Output("plot", "figure"),
        Output("slct_filter_column", "options"),
        Output("slct_specific_values", "options"),
        Output("slct_exact_value", "options")
    ],
    [
        Input("x-axis", "value"),
        Input("y-axis", "value"),
        Input("slct_filter_column", "value"),
        Input("slct_specific_values", "value"),
        Input("slct_exact_value", "value"),
        Input("slct_plot_type", "value")
    ]
)
def generate_Plot(x, y, color, values, exact_val, plot):
    if color == None:
        color = ''
    dff = df.copy()
    exact_values = list()
    dfa = df.copy()
    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
        dfa = dfa[dfa[y[0]]!='else']
    ss = dfa[y[0]][~dfa.duplicated(y[0])]
    for d in ss:
        exact_values.append({'value':d, 'label':d})
    if len(exact_val) >= 1:
        dff = dff[dff[y[0]].isin(exact_val)]
    colorOp = list()
    valueOp = list()
    li = ['Year', 'Month', 'Trimestre', 'Continent', 'Country', 'Region', 'Industry', 'Category Type', 'Category Name', 'Skills', 'Employers', 'Jobs', 'Category Value', 'Rank']
    for i in li:
        if (i != x) & (i not in y):
            colorOp.append({'label':i, 'value':i})
    Specific = list()
    if color == 'Year':
        Specific = ['Trimestre', 'Month']
    elif color == 'Trimestre':
        Specific = ['Month']
    elif color == 'Continent':
        Specific = ['Country']
    elif color == 'Country':
        Specific = ['Region']
    elif color == 'Category Type':
        Specific = ['Category Name']
    elif color == 'Category Name':
         Specific = ['Skills', 'Employers', 'Jobs']
    else:
        Specific = [color]
############################    Sunburst Chart      ############################
    if plot == 'Sunburst Chart':
        t = list(y)
        if x not in y:
            t.append(x)
        if color == '':
            fig = px.sunburst(dff, path=t)
        else:
            fig = px.sunburst(dff, path=t)
        fig.update_layout(
            margin = dict(t=10, l=10, r=10, b=10)
        )
############################    Line Chart      ############################
    elif plot == 'Line Chart':
        if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):            
            dff = dff[dff[y[0]]!='else']
        if color == '':
            fig = px.line(dff, x=y[0], y=x, markers=True)
        else:
            if len(values) != 0:
                if ('Trimestre 1' in values) | ('Trimestre 2' in values) | ('Trimestre 3' in values):
                    dff = dff[dff['Trimestre'].isin(values)]
                if ('Jan' in values) | ('Feb' in values) | ('Mar' in values) | ('Apr' in values) | ('May' in values) | ('Jun' in values) | ('Jul' in values) | ('Aug' in values) | ('Sep' in values) | ('Oct' in values) | ('Nov' in values) | ('Dec' in values):
                    dff = dff[dff['Month'].isin(values)]
            fig = px.line(dff, x=y[0], y=x, markers=True, color=color)
    elif plot == 'Bar Chart':
        dfa = dff.copy()
        if ('Skills' in y[0]) | ('Jobs' in y[0]) | ('Employers' in y[0]):
            dfa = dfa[dfa[y[0]]!='else']
        if color == '':
            fig = px.bar(dfa, x=y[0], y=x)
            print("Goood1")
        else:
            if len(values) != 0:
                dfa = dfa[dfa[Specific[0]].isin(values)]
            fig = px.bar(dfa, x=y[0], y=x, color=color, text=color, barmode='group')
            print("Goood2")
        fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',marker_line_width=1.5, opacity=0.6)
                  
    else:
        fig = go.Figure()
        fig.update_yaxes(title_text=x)
############################    Box Plot      ############################
        if plot == 'Box Plot':
            if color == '':
                for k in range(len(y)):
                    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                        dfa = dff.copy()
                        dfa = dfa[dfa[y[k]]!='else']
                        fig.add_trace(go.Box(x=dfa[y[k]], y=dff[x], name=y[k]))
                    else:
                        fig.add_trace(go.Box(x=dff[y[k]], y=dff[x], name=y[k]))
            else:
                if len(values) == 0:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                                dfa = dff.copy()
                                dfa = dfa[dfa[y[k]]!='else']
                                fig.add_trace(go.Box(x=dfa[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                            else:
                                fig.add_trace(go.Box(x=dfg[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                else:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            for s in range(len(Specific)):
                                dfk = dfg[dfg[Specific[s]].isin(values)]
                                for Val, dfe in dfk.groupby(Specific[s]):
                                    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                                        dfa = dff.copy()
                                        dfa = dfa[dfa[y[k]]!='else']
                                        fig.add_trace(go.Box(x=dfa[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
                                    else:
                                        fig.add_trace(go.Box(x=dfe[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
############################    Bar Chart      ############################
        ### Just trying ####
#         if plot == 'Bar Chart':
#             fig.update_layout(barmode='group')
#             if color == '':
#                 for k in range(len(y)):
#                     if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
#                         dfa = dff.copy()
#                         dfa = dfa[dfa[y[k]]!='else']
#                         fig.add_trace(go.Bar(x=dfa[y[k]], y=dff[x], name=y[k]))
#                     else:
#                         fig.add_trace(go.Bar(x=dff[y[k]], y=dff[x], name=y[k]))
#             else:
#                 if len(values) == 0:
#                     for k in range(len(y)):
#                         for colors, dfg in dff.groupby(color):
#                             if ('Skills' in x) | ('Jobs' in x) | ('Employers' in x):
#                                 dfa = dfg.copy()
#                                 dfa = dfa[dfa[y[k]]!='else']
#                                 fig.add_trace(go.Bar(x=dfa[y[k]], y=dfg[x], name=y[k]+' per '+colors))
#                             else:
#                                 fig.add_trace(go.Bar(x=dfg[y[k]], y=dfg[x], name=y[k]+' per '+colors))
#                 else:
#                     for k in range(len(y)):
#                         for colors, dfg in dff.groupby(color):
#                             for s in range(len(Specific)):
#                                 dfk = dfg[dfg[Specific[s]].isin(values)]
#                                 for Val, dfe in dfk.groupby(Specific[s]):
#                                     if ('Skills' in x) | ('Jobs' in x) | ('Employers' in x):
#                                         dfa = dfe.copy()
#                                         dfa = dfa[dfa[y[k]]!='else']
#                                         fig.add_trace(go.Bar(x=dfa[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
#                                     else:
#                                         fig.add_trace(go.Bar(x=dfe[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
#             fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',marker_line_width=1.5, opacity=0.6)
############################    Histogram      ############################
        if plot == 'Histogram':
            if color == '':
                for k in range(len(y)):
                    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                        dfa = dff.copy()
                        dfa = dfa[dfa[y[k]]!='else']
                        fig.add_trace(go.Histogram(x=dfa[y[k]], y=dff[x], name=y[k]))
                    else:
                        fig.add_trace(go.Histogram(x=dff[y[k]], y=dff[x], name=y[k]))
            else:
                if len(values) == 0:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            if ('Skills' in x) | ('Jobs' in x) | ('Employers' in x):
                                dfa = dfg.copy()
                                dfa = dfa[dfa[y[k]]!='else']
                                fig.add_trace(go.Histogram(x=dfa[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                            else:
                                fig.add_trace(go.Histogram(x=dfg[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                else:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            for s in range(len(Specific)):
                                dfk = dfg[dfg[Specific[s]].isin(values)]
                                for Val, dfe in dfk.groupby(Specific[s]):
                                    if ('Skills' in x) | ('Jobs' in x) | ('Employers' in x):
                                        dfa = dfe.copy()
                                        dfa = dfa[dfa[y[k]]!='else']
                                        fig.add_trace(go.Histogram(x=dfa[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
                                    else:
                                        fig.add_trace(go.Histogram(x=dfe[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
############################    Scatter Plot      ############################
        if plot == 'Scatter Plot':
            if color == '':
                for k in range(len(y)):
                    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                        dfa = dff.copy()
                        dfa = dfa[dfa[y[k]]!='else']
                        fig.add_trace(go.Scatter(x=dfa[y[k]], y=dff[x], name=y[k]))
                    else:
                        fig.add_trace(go.Scatter(x=dff[y[k]], y=dff[x], name=y[k]))
            else:
                if len(values) == 0:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                                dfa = dff.copy()
                                dfa = dfa[dfa[y[k]]!='else']
                                fig.add_trace(go.Scatter(x=dfa[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                            else:
                                fig.add_trace(go.Scatter(x=dfg[y[k]], y=dfg[x], name=y[k]+' per '+colors))
                else:
                    for k in range(len(y)):
                        for colors, dfg in dff.groupby(color):
                            for s in range(len(Specific)):
                                dfk = dfg[dfg[Specific[s]].isin(values)]
                                for Val, dfe in dfk.groupby(Specific[s]):
                                    if ('Skills' in y) | ('Jobs' in y) | ('Employers' in y):
                                        dfa = dff.copy()
                                        dfa = dfa[dfa[y[k]]!='else']
                                        fig.add_trace(go.Scatter(x=dfa[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
                                    else:
                                        fig.add_trace(go.Scatter(x=dfe[y[k]], y=dfe[x], name=y[k]+' per '+colors+' in '+Val))
    if color != '':
        for q in range(len(Specific)):
            res = df[Specific[q]][~df.duplicated(Specific[q])]
            for p in res:
                valueOp.append({'value': p, 'label': p})
                                  
    return fig, colorOp, valueOp, exact_values

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
Goood1
Goood2
Goood2
